In [1]:
from gurobipy import *
import numpy  as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")

In [12]:
m = Model("MasterProblem")

# Create Variables 
horizon      = 20 # Time steps
upper_bound  = 10 # Uppper threshold for water 
link_up = horizon*upper_bound

upper_volume = 100 # Volume limits.

basins  = [i for i in range(0, 3)]

# Total inflow into nodes in that time 
inflows = [20, 10, 10]
# Inital volume 
init_vol = [80, 90, 90]

# Create Basin Volume
volume_basins = m.addVars(basins,
                          lb=0,
                          ub=upper_volume,
                          vtype=GRB.CONTINUOUS,
                          name="Volume_Basin")
# Create Volume Exchange 
volume_links  = m.addVars(basins,
                          lb=0,
                          ub=link_up,
                          vtype=GRB.CONTINUOUS,
                          name="Volume_Link")

In [13]:
# Constraints 
## 1. Link and basin upper bound constraint set in variable. 
## 2. Mass Balance constraint 
m.addConstr(volume_basins[0] == init_vol[0] + inflows[0] - volume_links[0], "P0")
m.addConstr(volume_basins[1] == init_vol[1] + inflows[1] - volume_links[1], "P1")
m.addConstr(volume_basins[2] == init_vol[2] + inflows[2] + volume_links[0] + volume_links[1] - volume_links[2], "P1")

<gurobi.Constr *Awaiting Model Update*>

In [14]:
m.setObjective(volume_basins.sum(), GRB.MINIMIZE)

In [15]:
m.optimize()

Optimize a model with 3 rows, 6 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+02, 2e+02]
  RHS range        [1e+02, 1e+02]
Presolve removed 3 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  1.000000000e+02


In [16]:
for v in m.getVars():
    print(v.varName, v.x)

Volume_Basin[0] 0.0
Volume_Basin[1] 0.0
Volume_Basin[2] 100.0
Volume_Link[0] 100.0
Volume_Link[1] 100.0
Volume_Link[2] 200.0
